In [2]:
import os
import glob
from functools import reduce
import operator

In [3]:
for file in glob.glob('stl_files/*.stl'):
  print(file)

stl_files/cube.stl
stl_files/largecube.stl
stl_files/monkey.stl


In [58]:
import itertools as it
import numpy as np
import pandas as pd
from subprocess import check_output,CalledProcessError

"""
this script automates the process of slicing the same stl file with many possible combinations of command line arguments 
that can be passed to slic3r
"""

#dict of lists of possible arguments to be tested
configurations = {"--nozzle-diameter":[0.1,
                                       0.5,
                                       1.0
                                      ],                  
                  #"--use-firmware-retraction":["no","yes"],
                  #"--use-volumetric-e":["no","yes"], 
                  #"--gcode-arcs":["no","yes"], 
                  #"--vibration-limit":[0,5,10], 
                  #"--pressure-advance":[], 
                  #"--filament-diameter":[],
                  #"--extrusion-multiplier":[],
                  #"--temperature":[],
                  #"--first-layer-temperature":[],
                  #"--bed-temperature":[],
                  #"--first-layer-bed-temperature":[],
                  #"--travel-speed":[], 
                  #"--perimeter-speed":[], 
                  #"--small-perimeter-speed":[], 
                  #"--external-perimeter-speed":[], 
                  #"--infill-speed":[], 
                  #"--solid-infill-speed":[], 
                  #"--top-solid-infill-speed":[], 
                  #"--support-material-speed":[], 
                  #"--support-material-interface-speed":[], 
                  #"--bridge-speed":[], 
                  #"--gap-fill-speed":[], 
                  #"--first-layer-speed":[], 
                  #"--perimeter-acceleration":[],
                  #"--infill-acceleration":[],
                  #"--bridge-acceleration":[],
                  #"-first-layer-acceleration":[],
                  #"--default-acceleration":[],
                  #"--layer-height":[],
                  #"--first-layer-height":[],
                  #"--infill-every-layers":[],
                  #"--solid-infill-every-layers":[],
                  #"--perimeters":[],
                  #"--top-solid-layers":[],
                  #"--bottom-solid-layers":[],
                  #"--solid-layers":[],
                  #"--fill-density":[],
                  #"--fill-angle":[],
                  #"--fill-pattern":[],
                  #"--external-fill-pattern":[],                  
                  #"--end-gcode":[],
                  #"--before-layer-gcode":[],
                  #"--layer-gcode":[],
                  #"--toolchange-gcode":[],
                  #"--seam-position":[],
                  #"--external-perimeters-first":[],
                  #"--spiral-vase":[],
                  #"--only-retract-when-crossing-perimeters":[],
                  #"--solid-infill-below-area":[],
                  #"--infill-only-where-needed":[],
                  #"--infill-first":[],
                  #"--extra-perimeters":[],
                  #"--avoid-crossing-perimeters":[],
                  #"--thin-walls":[],
                  #"--overhangs":[],
                  #"--support-material":[],
                  #"--support-material-threshold":[],
                  #"--support-material-pattern":[],
                  #"--support-material-spacing":[],
                  #"--support-material-angle":[],
                  #"--support-material-contact-distance":[],
                  #"--support-material-interface-layers":[],
                  #"--support-material-interface-spacing":[],
                  #"--raft-layers":[],
                  #"--support-material-enforce-layers":[],
                  #"--dont-support-bridges":[],
                  #"--retract-length":[],
                  #"--retract-speed":[],
                  #"--retract-restart-extra":[],
                  #"--retract-before-travel":[],
                  #"--retract-lift":[],
                  #"--retract-layer-change":[],
                  #"--wipe":[],
                  #"--cooling":[],
                  #"--min-fan-speed":[],
                  #"--max-fan-speed":[],
                  #"--bridge-fan-speed":[],
                  #"--fan-below-layer-time":[],
                  #"--slowdown-below-layer-time":[],
                  #"--min-print-speed":[],
                  #"--disable-fan-first-layers":[],
                  #"--fan-always-on":[],
                  #"--skirts":[],
                  #"--skirt-distance":[],
                  #"--skirt-height":[],
                  #"--min-skirt-length":[],
                  #"--brim-width":[],
                  #"--scale":[],
                  #"--rotate":[],
                  #"--duplicate":[],
                  #"--duplicate-grid":[],
                  #"--duplicate-distance":[],
                  #"--xy-size-compensation":[],
                  #"--complete-objects":[],
                  #"--extruder-clearance-radius":[],
                  #"--extruder-clearance-height":[],
                  #"--notes":[],
                  #"--resolution":[],
                  #"--extrusion-width":[],
                  #"--first-layer-extrusion-width":[],
                  #"--perimeter-extrusion-width Set a different extrusion width for perimeters":[],
                  #"--external-perimeter-extrusion-width":[],
                  #"--infill-extrusion-width":[],
                  #"--solid-infill-extrusion-width":[],
                  #"--top-infill-extrusion-width":[],
                  #"--support-material-extrusion-width":[],
                  #"--infill-overlap":[],
                  #"--bridge-flow-ratio Multiplier for extrusion when bridging (> 0, default: 1)":[],
                  #"--extruder-offset":[],
                  #"--perimeter-extruder":[],
                  #"--infill-extruder":[],
                  #"--solid-infill-extruder":[],
                  #"--support-material-extruder":[],
                  #"--support-material-interface-extruder Extruder to use for support material interface (1+, default: 1)":[],
                  #"--ooze-prevention":[],
                  #"--standby-temperature-delta":[],
                  #"--ooze-prevention is enabled (default: -5)":[], 
                  #"--retract-length-toolchange":[],
                  #"--retract-restart-extra-toolchange":[]                  
                 }


combinations = it.product(*(configurations[Name] for Name in configurations))
count=1
total=len(list(combinations))
#loop through every combination of arguments
for configuration in list(it.product(*configurations.values())):
    #construct a list of command line arguments   
    
    cmd=[]
    for key,value in zip(configurations.keys(),configuration):
      if value:
        cmd.append(key)
        cmd.append(value)
        
    print('%d out of %d' % (count,total))
    cmd_str='slic3r'
    for arg in cmd:
      cmd_str += ' '+str(arg)
      
    cmd_str+=' ./stl_files/largecube.stl'
    print(cmd_str)
    
    #run the command and error if the exit status is non-zero
    check_output(cmd_str)
    print('done')
    count+=1


1 out of 3
slic3r --nozzle-diameter 0.1 ./stl_files/largecube.stl


FileNotFoundError: [Errno 2] No such file or directory: 'slic3r --nozzle-diameter 0.1 ./stl_files/largecube.stl': 'slic3r --nozzle-diameter 0.1 ./stl_files/largecube.stl'

In [54]:
os.getcwd()

'/home/thom/PycharmProjects/optimization_of_3d_printable_objects'